In [1]:
import csv
import time

from datetime import datetime, timedelta
from novaclient import client
# from kafka import KafkaProducer
import os

from novaclient import client
from keystoneauth1 import session
from keystoneauth1.identity import v3

In [2]:
# def get_file_and_topic_names(project_name, current_date):
    # if 'prd' in project_name.lower():
    #     suffix = 'prd'
    # elif 'stg' in project_name.lower():
    #     suffix = 'stg'
    # else:
    #     suffix = 'prd'  # 기본적으로 'prd'로 설정

def get_file_and_topic_names(current_date):
    file_name = f'C://Beomjun//csv//eu_adminrc_{current_date}.csv'
    # topic_name = f'EU_VM_2_{suffix.upper()}'
    return file_name

In [4]:
def nova_extract(project_name):
    current_date = datetime.now().strftime("%Y%m%d")
    
    auth = v3.Password(
        auth_url='http://cloud-control-vip.eu-central.openstack.h53:5000/v3',
        username='admin',
        password='TldhTl1!',
        user_domain_name='Default',
        project_name='admin',
        project_domain_name='Default'
    )
    
    sess = session.Session(auth=auth)
    nova = client.Client('2.1', session=sess)
    
    # 모든 프로젝트 VM 가져오기
    vms = nova.servers.list(search_opts={'all_tenants': 1})

    csv_file_path = get_file_and_topic_names(current_date)
    print("csv_file_name:", csv_file_path)
    csv_header = ["id", "name", "status", "flavor", "address", "availability_zone", "hostname", "created_at", "updated_at"]

    file_exists = os.path.exists(csv_file_path)
    with open(csv_file_path, mode='a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        if not file_exists:
            writer.writerow(csv_header)

        for vm in vms:
            addresses_info = ""
            for key, values in vm.addresses.items():
                for addr_info in values:
                    addresses_info += f'{key}:{addr_info["addr"]}\n'
            addresses_info = addresses_info.strip()

            flavor_id = vm.flavor['id']
            flavor = nova.flavors.get(flavor_id)

            try:
                flavor = nova.flavors.get(flavor_id)
                flavor_name = flavor.name
            except Exception as e:
                flaver_name = ""

            vm_data = [
                vm.id, vm.name, vm.status, flavor.name, addresses_info,
                vm._info['OS-EXT-AZ:availability_zone'], vm._info['OS-EXT-SRV-ATTR:host'],
                vm.created, vm.updated
            ]
            writer.writerow(vm_data)
    return csv_file_path
    
def cleanup(tenants):
#       schedule.run_pending()
       time.sleep(1)

       current_date = datetime.now().strftime("%Y%m%d")
       retention_period = 5

       delete_date = (datetime.now() - timedelta(days=retention_period)).strftime("%Y%m%d")

       host_dir = './csv'
       file_list = os.listdir(host_dir)

       for tenant in tenants:
               for file_name in file_list:
                       if file_name.startswith(f'vm_eu_2_{tenant}_') and file_name <= f'vm_eu_2_{tenant}_{delete_date}.csv':
                           file_path = os.path.join(host_dir, file_name)
                           os.remove(file_path)
                           print("file remove success!")


# 프로젝트 이름 리스트
# project_names = ['B2C_PRD', 'B2C_STG']
project_names = ['admin']
# tenants = ['prd', 'stg']
# topic_files = {}

# 각 프로젝트에 대해 함수 실행
for project_name in project_names:
    print("This project is", project_name)
    csv_file_path = nova_extract(project_name)

# cleanup(tenants)

This project is admin
csv_file_name: C://Beomjun//csv//eu_adminrc_20250502.csv
